# Практическая работа №6

# Выполнил студент группы ББМО-01-23 Морин А.А.


### Шаг 1. Загрузка и создание двух различных моделей

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

model1.fit(train_images, train_labels, epochs=5)

model1.save('mnist_model_1.h5')

# Модель 2: Свёрточная нейронная сеть
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)

model2.save('mnist_model_2.h5')

d:\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 987us/step - accuracy: 0.8801 - loss: 0.4385
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 941us/step - accuracy: 0.9638 - loss: 0.1253
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 928us/step - accuracy: 0.9744 - loss: 0.0833
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 917us/step - accuracy: 0.9834 - loss: 0.0567
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 931us/step - accuracy: 0.9868 - loss: 0.0435


d:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9098 - loss: 0.3033
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9839 - loss: 0.0532
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9905 - loss: 0.0323
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9933 - loss: 0.0216
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9956 - loss: 0.0140


### Шаг 2: Реализация атаки FGSM на первую модель

In [2]:
import numpy as np

def fgsm_attack(image, epsilon, gradient):

  perturbed_image = image + epsilon * np.sign(gradient)
  perturbed_image = np.clip(perturbed_image, 0, 1)
  return perturbed_image

def generate_adversarial_example(model, image, label, epsilon):

    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))

    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)

    gradient = tape.gradient(loss, image)


    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())

    return np.reshape(adversarial_image, (28, 28, 1))

def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))

    adversarial_images = np.array(adversarial_images)

    print("Shape of adversarial_images:", adversarial_images.shape)

    return adversarial_images


# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)


Shape of adversarial_images: (10000, 28, 28)


### Шаг 3. Оценка противоречивых примеров на обеих моделях

In [3]:
# Оценка первой модели на противоречивых примерах
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model_1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model_2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.0896 - loss: 6.5756
Accuracy of model_1 on adversarial examples: 0.11909999698400497
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9603 - loss: 0.1358
Accuracy of model_2 on adversarial examples from model1: 0.9670000076293945


### Шаг 4: Анализ переносимости атак

In [4]:
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.9235 - loss: 0.2604
Accuracy of model1 on adversarial examples from model2: 0.9377999901771545


### Вывод:
В ходе выполнения работы было исследовано воздействие атаки по переносу, при которой противоречивые примеры, сгенерированные для одной модели, использовались для атаки на другую. Для реализации атаки использовался метод FGSM

В результате было выявлено, что модель 1, на которой были сгенерированы противоречивые примеры с использованием FGSM, продемонстрировала ощутимое падение точности с 98% до 8%, что свидетельствует о её высокой уязвимости к данной атаке. Модель 2 показала гораздо меньший эффект от атаки — её точность снизилась лишь на ~3%, что указывает на большую устойчивость к переносу атак, созданных для другой модели.

Дополнительно были сгенерированы противоречивые примеры с использованием FGSM для модели 2, и они были протестированы на модели 1. В этом случае точность модели 1 также снизилась, но падение составляет около 5%.

Атака по переносу с использованием FGSM может существенно ослабить точность модели, особенно если примеры сгенерированы для неё самой. Другая модель, показавшая более высокую устойчивость, остаётся менее подверженной таким атакам.